### PRI - Pipeline

In order to convert the dataset obtained from kaggle into a more usable format we will convert it into a SQLite database.

#### First we developed the database schema to better understand our goal with this step

The schema can be found in [this](movie_corpus_schema.sql) file.

Consist in 6 tables:
 - Movie: contains the movie title, release year, IMDB rating and votes. (poster, revenue, runtime)
 - Character: contains character name, the movie it belongs to, its gender and credits position, that determine its relevance.
 - Genre: contains the genre name. Its linked to the movie through a pivot table called movie_genre.
 - Conversation: contains the ids of the caracters involved in the conversation and the id of the movie it belongs to.
 - Line: contains the text, the character that said it and the movie it belongs to. Also contains the conversation it was said in.

#### After developing the schema we can create the database file

In [1]:
import sqlite3
import os

# create database
if os.path.exists('final.db'):
    os.remove('final.db')

os.system("cat movie_corpus_schema.sql | sqlite3 final.db")

connection = sqlite3.connect('final.db')

Error: near line 44: in prepare, near "ON": syntax error (1)


#### After this step we will start converting the csv data into database data

Since we want to create a table with all the genres we need to parse it from the [movies](datasets/movie_titles_metadata.csv) format (e.g.: ['action' 'crime' 'drama' 'thriller']).

In [2]:
import pandas as pd

movies = pd.read_csv('datasets/movie_titles_metadata.csv')

# extract all genres from movies, the genres are stored as ['genre1' 'genre2' ...]
genres = set()
for genre in movies['genres']:
    genres.update(genre.split(' '))

# clean genres removing the ' and [ ]
genres = [genre.replace("'", "").replace("[", "").replace("]", "") for genre in genres]
# remove empty genres
genres = [genre for genre in genres if genre != '']
genres = set(genres)

# transform genres into a list of tuples (id, name)
genres = [(i + 1, genre) for i, genre in enumerate(genres)]

# create a table for genre
genres = pd.DataFrame(genres, columns=['id', 'name'])
genres.to_sql('genre', connection, if_exists='append', index=False)

24

#### Creating the movie_genre pivot table

Our goal is to populate this table with the movie and genre ids.

In [3]:
# create movie id and genre id pivot table
movie_genre = []
for i, row in movies.iterrows():
    for genre in row['genres'].split(' '):
        genre = genre.replace("'", "").replace("[", "").replace("]", "")
        if genre != '':
            genre_id = genres[genres['name'] == genre]['id'].values[0]
            movie_genre.append((row['id'], genre_id))
        
movie_genre = pd.DataFrame(movie_genre, columns=['movie_id', 'genre_id'])
movie_genre.to_sql('movie_genre', connection, if_exists='append', index=False)

1871

#### Populating movie table

After fetching the genres information we can drop this column from the dataframe and finally insert the relevant information into the table.

In [4]:
# drop movies genres column
movies = movies.drop(columns=['genres'])
movies = pd.DataFrame(movies, columns=["id","title","year","imdb_rating","imdb_votes"])

movies.to_sql('movie', connection, if_exists='append', index=False)

617

#### Populate the characters table

In this table the information available in the csv is all necessary, excluding the movie title. Since there are some characters with unknown gender or credits positions we will convert this cases to NULL values.

In [5]:
# create characters table
characters = pd.read_csv('./datasets/movie_characters_metadata.csv')
characters = characters.drop(columns=["movie_title"])

# replace ? with null
for i, character in characters.iterrows():
    if character['gender'] == '?':
        character['gender'] = None
    if character['credit_pos'] == '?':
        character['credit_pos'] = None


characters.to_sql('character', connection, if_exists='append', index=False)

9035

#### Populate line and conversation tables

To finish this database population we still need to populate these two tables. We need to convert the representation of conversations (e.g.: ['L271' 'L272' 'L273' 'L274' 'L275']) into foreign keys in the line table.

We also realized that the lines include text formatting tags so we will remove them.

In [6]:
conversations = pd.read_csv('./datasets/movie_conversations.csv')
lines = pd.read_csv('./datasets/movie_lines.csv')
lines.drop(columns=['character_name'], inplace=True)
# add column conversation_id to lines with default null
lines['conversation_id'] = None
conversations['id'] = conversations.index + 1

for i, line in lines.iterrows():
    line['line_text'] = str(line['line_text']).replace("<u>", "").replace("</u>", "").replace("<i>", "").replace("</i>", "").replace("<b>", "").replace("</b>", "")


for i, conversation in conversations.iterrows():
    conversation_id = i + 1
    lines_aux = []
    for line in conversation['lines'].split(' '):
        line = line.replace("'", "").replace("[", "").replace("]", "")
        if line != '':
            lines_aux.append(line)
    for line in lines_aux:
        lines.loc[lines['id'] == line, 'conversation_id'] = conversation_id


lines.to_sql('line', connection, if_exists='append', index=False)
conversations = conversations.drop(columns=['lines'])
conversations.to_sql('conversation', connection, if_exists='append', index=False)

connection.close()

#### Continued in [api fetching](api_test.ipynb) notebook